# Llama 3 한국어 성능 테스트하기 (금융, 법률 문서 기계독해 데이터셋)
## 작성자 : AISchool ( http://aischool.ai/%ec%98%a8%eb%9d%bc%ec%9d%b8-%ea%b0%95%ec%9d%98-%ec%b9%b4%ed%85%8c%ea%b3%a0%eb%a6%ac/ )
## Reference : https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct
## 금융, 법률 문서 기계독해 데이터 : https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=data&dataSetSn=71610

# 라이브러리 설치

In [ ]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 5.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

## 금융, 법률 문서 기계독해 데이터 불러오기

In [ ]:
# Sample/02.라벨링데이터/1.정답경계 추출형/TeL_정답경계_2022_sample.json

In [ ]:
import json

# JSON 파일을 열고 데이터를 로드합니다.
with open('TeL_정답경계_2022_sample.json', 'r', encoding='utf-8-sig') as file:
    data = json.load(file)

In [ ]:
data

{'Dataset': {'Identifier': 'TEXT_QnA_FinancialLaw_01',
  'name': '금융법률 문서 기계독해 데이터',
  'src_path': '/dataSet/text/',
  'label_path': '/dataSet/text/',
  'category': 2,
  'type': 0},
 'data': [{'doc_id': 'TNL041227',
   'doc_title': '신 미·일방위협력지침의 주요내용 및 쟁점과 한국의 대응과제',
   'doc_source': '국회입법조사처',
   'doc_published': 20150603,
   'created': '20221219143503',
   'doc_class': {'class': '법률/사회과학/법학일반', 'code': '연구보고서'},
   'paragraphs': [{'context_id': '2356574',
     'context': '1. 들어가며\n지난 2015년 4월 27일 아베신조(安倍晋三) 총리의 방미를 계기로 미·일 양국은 ‘신 미·일방위협력지침’에 합의하였다. 이는 미·일 간의 안보협력관계를 체계적으로 실행하기 위해 1978년에 마련되고, 1997년에 개정된 ‘미·일방위협력지침’을 대체하는 것으로 미·일 동맹의 군사적 활동 공간을 전 세계로 확장하는 내용을 담고 있다.\n이에 대해서 중국은 자국의 이익을 훼손할 수 있다는 점에서 비판적인 입장을 피력했는데, 실제로 아베 총리는 미·일 동맹 강화가 북한뿐만 아니라 중국의 위협에 대응하기 위한 것이라고 발언함으로써 동북아의 평화안보질서에 부정적인 영향을 미치게 될 것이라는 주장에 힘이 실리고 있다.',
     'tbs': None,
     'qas': [{'qa_type': 1,
       'question_id': '6379347',
       'question': '처음으로 미·일방위협력지침이 정해진 건 언제야',
       'answer': {'source': '2356574',

In [ ]:
# 추출할 데이터를 저장할 리스트
extracted_data = []

# 데이터 내에서 각 문단과 질문, 답변 텍스트를 추출하여 딕셔너리로 저장
for item in data["data"]:
    for paragraph in item["paragraphs"]:
        context = paragraph["context"]
        for qa in paragraph["qas"]:
            question = qa["question"]
            answer_text = qa["answer"]["text"]
            # 딕셔너리 생성 및 리스트에 추가
            extracted_data.append({
                "context": context,
                "question": question,
                "answer": answer_text
            })

In [ ]:
extracted_data[0]

{'context': '1. 들어가며\n지난 2015년 4월 27일 아베신조(安倍晋三) 총리의 방미를 계기로 미·일 양국은 ‘신 미·일방위협력지침’에 합의하였다. 이는 미·일 간의 안보협력관계를 체계적으로 실행하기 위해 1978년에 마련되고, 1997년에 개정된 ‘미·일방위협력지침’을 대체하는 것으로 미·일 동맹의 군사적 활동 공간을 전 세계로 확장하는 내용을 담고 있다.\n이에 대해서 중국은 자국의 이익을 훼손할 수 있다는 점에서 비판적인 입장을 피력했는데, 실제로 아베 총리는 미·일 동맹 강화가 북한뿐만 아니라 중국의 위협에 대응하기 위한 것이라고 발언함으로써 동북아의 평화안보질서에 부정적인 영향을 미치게 될 것이라는 주장에 힘이 실리고 있다.',
 'question': '처음으로 미·일방위협력지침이 정해진 건 언제야',
 'answer': '1978년'}

In [ ]:
print('전체 Question-Answer 개수 :', len(extracted_data))

전체 Question-Answer 개수 : 10


In [ ]:
extracted_data[0]['context']

'1. 들어가며\n지난 2015년 4월 27일 아베신조(安倍晋三) 총리의 방미를 계기로 미·일 양국은 ‘신 미·일방위협력지침’에 합의하였다. 이는 미·일 간의 안보협력관계를 체계적으로 실행하기 위해 1978년에 마련되고, 1997년에 개정된 ‘미·일방위협력지침’을 대체하는 것으로 미·일 동맹의 군사적 활동 공간을 전 세계로 확장하는 내용을 담고 있다.\n이에 대해서 중국은 자국의 이익을 훼손할 수 있다는 점에서 비판적인 입장을 피력했는데, 실제로 아베 총리는 미·일 동맹 강화가 북한뿐만 아니라 중국의 위협에 대응하기 위한 것이라고 발언함으로써 동북아의 평화안보질서에 부정적인 영향을 미치게 될 것이라는 주장에 힘이 실리고 있다.'

In [ ]:
extracted_data[0]['question']

'처음으로 미·일방위협력지침이 정해진 건 언제야'

In [ ]:
extracted_data[0]['answer']

'1978년'

In [ ]:
extracted_data[0]

{'context': '1. 들어가며\n지난 2015년 4월 27일 아베신조(安倍晋三) 총리의 방미를 계기로 미·일 양국은 ‘신 미·일방위협력지침’에 합의하였다. 이는 미·일 간의 안보협력관계를 체계적으로 실행하기 위해 1978년에 마련되고, 1997년에 개정된 ‘미·일방위협력지침’을 대체하는 것으로 미·일 동맹의 군사적 활동 공간을 전 세계로 확장하는 내용을 담고 있다.\n이에 대해서 중국은 자국의 이익을 훼손할 수 있다는 점에서 비판적인 입장을 피력했는데, 실제로 아베 총리는 미·일 동맹 강화가 북한뿐만 아니라 중국의 위협에 대응하기 위한 것이라고 발언함으로써 동북아의 평화안보질서에 부정적인 영향을 미치게 될 것이라는 주장에 힘이 실리고 있다.',
 'question': '처음으로 미·일방위협력지침이 정해진 건 언제야',
 'answer': '1978년'}

In [ ]:
def format_context_question(data):
    """
    Extracts 'context' and 'question' from a dictionary and formats them into a string.

    Parameters:
    data (dict): The dictionary containing 'context', 'question', and 'answer'.

    Returns:
    str: A formatted string containing the 'context' and 'question'.
    """
    # 'context'와 'question'을 추출하여 문자열로 반환
    return f"Context: {data['context']}\nQuestion: {data['question']}"

In [ ]:
print(format_context_question(extracted_data[0]))

Context: 1. 들어가며
지난 2015년 4월 27일 아베신조(安倍晋三) 총리의 방미를 계기로 미·일 양국은 ‘신 미·일방위협력지침’에 합의하였다. 이는 미·일 간의 안보협력관계를 체계적으로 실행하기 위해 1978년에 마련되고, 1997년에 개정된 ‘미·일방위협력지침’을 대체하는 것으로 미·일 동맹의 군사적 활동 공간을 전 세계로 확장하는 내용을 담고 있다.
이에 대해서 중국은 자국의 이익을 훼손할 수 있다는 점에서 비판적인 입장을 피력했는데, 실제로 아베 총리는 미·일 동맹 강화가 북한뿐만 아니라 중국의 위협에 대응하기 위한 것이라고 발언함으로써 동북아의 평화안보질서에 부정적인 영향을 미치게 될 것이라는 주장에 힘이 실리고 있다.
Question: 처음으로 미·일방위협력지침이 정해진 건 언제야


# Llama 3 모델 불러오기

In [ ]:
import os
os.environ['HF_TOKEN']="여러분의 HuggingFace Access Token값"

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [ ]:
def generate_response(system_message, user_message):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message},
    ]

    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = model.generate(
        input_ids,
        max_new_tokens=256,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9
    )
    response = outputs[0][input_ids.shape[-1]:]

    return tokenizer.decode(response, skip_special_tokens=True)

In [ ]:
print(model.device)

cuda:0


# Llama 3의 금융, 법률 문서 기계독해 성능을 정성적으로 측정해보기

In [ ]:
extracted_data[0]

{'context': '1. 들어가며\n지난 2015년 4월 27일 아베신조(安倍晋三) 총리의 방미를 계기로 미·일 양국은 ‘신 미·일방위협력지침’에 합의하였다. 이는 미·일 간의 안보협력관계를 체계적으로 실행하기 위해 1978년에 마련되고, 1997년에 개정된 ‘미·일방위협력지침’을 대체하는 것으로 미·일 동맹의 군사적 활동 공간을 전 세계로 확장하는 내용을 담고 있다.\n이에 대해서 중국은 자국의 이익을 훼손할 수 있다는 점에서 비판적인 입장을 피력했는데, 실제로 아베 총리는 미·일 동맹 강화가 북한뿐만 아니라 중국의 위협에 대응하기 위한 것이라고 발언함으로써 동북아의 평화안보질서에 부정적인 영향을 미치게 될 것이라는 주장에 힘이 실리고 있다.',
 'question': '처음으로 미·일방위협력지침이 정해진 건 언제야',
 'answer': '1978년'}

In [ ]:
질문_prompt = format_context_question(extracted_data[0])
질문_prompt

'Context: 1. 들어가며\n지난 2015년 4월 27일 아베신조(安倍晋三) 총리의 방미를 계기로 미·일 양국은 ‘신 미·일방위협력지침’에 합의하였다. 이는 미·일 간의 안보협력관계를 체계적으로 실행하기 위해 1978년에 마련되고, 1997년에 개정된 ‘미·일방위협력지침’을 대체하는 것으로 미·일 동맹의 군사적 활동 공간을 전 세계로 확장하는 내용을 담고 있다.\n이에 대해서 중국은 자국의 이익을 훼손할 수 있다는 점에서 비판적인 입장을 피력했는데, 실제로 아베 총리는 미·일 동맹 강화가 북한뿐만 아니라 중국의 위협에 대응하기 위한 것이라고 발언함으로써 동북아의 평화안보질서에 부정적인 영향을 미치게 될 것이라는 주장에 힘이 실리고 있다.\nQuestion: 처음으로 미·일방위협력지침이 정해진 건 언제야'

In [ ]:
llama3_inference_result = generate_response(system_message="너는 주어진 Context를 토대로 Question에 답변하는 챗봇이야. Question에 대한 답변을 한국어로 답해줘.",
                             user_message=질문_prompt)
print(llama3_inference_result)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


1978년에 미·일방위협력지침이 처음으로 정해졌습니다.


In [ ]:
def print_text_comparison(ground_truth, llama, gpt4, gpt35, clova, gemini):
    # 각 소스와 텍스트를 출력합니다.
    print("Ground Truth:")
    print(ground_truth)
    print("\nLlama:")
    print(llama)
    print("\nGPT-4:")
    print(gpt4)
    print("\nGPT-3.5:")
    print(gpt35)
    print("\nClova:")
    print(clova)
    print("\nGemini:")
    print(gemini)

In [ ]:
# 대표 LLM별 번역 추론 성능 비교
ground_truth_text = extracted_data[0]['answer']
llama_text = llama3_inference_result
gpt4_text = "처음으로 미·일방위협력지침이 정해진 건 1978년이야."
gpt35_text = "미·일방위협력지침이 처음으로 정해진 건 1978년이야."
clova_text = "미·일방위협력지침이 처음으로 정해진 것은 1978년입니다. 이후 1997년에 개정되었고, 2015년에 '신 미·일방위협력지침'에 합의하여 미·일 동맹의 군사적 활동 공간을 전 세계로 확장하였습니다."
gemini_text = "처음으로 미·일 방위협력지침이 정해진 것은 1978년입니다."

# Call the function to display the example texts
print_text_comparison(ground_truth_text, llama_text, gpt4_text, gpt35_text, clova_text, gemini_text)

Ground Truth:
1978년

Llama:
1978년에 미·일방위협력지침이 처음으로 정해졌습니다.

GPT-4:
처음으로 미·일방위협력지침이 정해진 건 1978년이야.

GPT-3.5:
미·일방위협력지침이 처음으로 정해진 건 1978년이야.

Clova:
미·일방위협력지침이 처음으로 정해진 것은 1978년입니다. 이후 1997년에 개정되었고, 2015년에 '신 미·일방위협력지침'에 합의하여 미·일 동맹의 군사적 활동 공간을 전 세계로 확장하였습니다.

Gemini:
처음으로 미·일 방위협력지침이 정해진 것은 1978년입니다.


In [ ]:
extracted_data[1]

{'context': '1. 들어가며\n지난 2015년 4월 27일 아베신조(安倍晋三) 총리의 방미를 계기로 미·일 양국은 ‘신 미·일방위협력지침’에 합의하였다. 이는 미·일 간의 안보협력관계를 체계적으로 실행하기 위해 1978년에 마련되고, 1997년에 개정된 ‘미·일방위협력지침’을 대체하는 것으로 미·일 동맹의 군사적 활동 공간을 전 세계로 확장하는 내용을 담고 있다.\n이에 대해서 중국은 자국의 이익을 훼손할 수 있다는 점에서 비판적인 입장을 피력했는데, 실제로 아베 총리는 미·일 동맹 강화가 북한뿐만 아니라 중국의 위협에 대응하기 위한 것이라고 발언함으로써 동북아의 평화안보질서에 부정적인 영향을 미치게 될 것이라는 주장에 힘이 실리고 있다.',
 'question': '언제 최초로 미국과 일본의 방위협력지침이 만들어졌어',
 'answer': '1978년'}

In [ ]:
질문_prompt = format_context_question(extracted_data[1])
질문_prompt

'Context: 1. 들어가며\n지난 2015년 4월 27일 아베신조(安倍晋三) 총리의 방미를 계기로 미·일 양국은 ‘신 미·일방위협력지침’에 합의하였다. 이는 미·일 간의 안보협력관계를 체계적으로 실행하기 위해 1978년에 마련되고, 1997년에 개정된 ‘미·일방위협력지침’을 대체하는 것으로 미·일 동맹의 군사적 활동 공간을 전 세계로 확장하는 내용을 담고 있다.\n이에 대해서 중국은 자국의 이익을 훼손할 수 있다는 점에서 비판적인 입장을 피력했는데, 실제로 아베 총리는 미·일 동맹 강화가 북한뿐만 아니라 중국의 위협에 대응하기 위한 것이라고 발언함으로써 동북아의 평화안보질서에 부정적인 영향을 미치게 될 것이라는 주장에 힘이 실리고 있다.\nQuestion: 언제 최초로 미국과 일본의 방위협력지침이 만들어졌어'

In [ ]:
llama3_inference_result = generate_response(system_message="너는 주어진 Context를 토대로 Question에 답변하는 챗봇이야. Question에 대한 답변을 한국어로 답해줘.",
                             user_message=질문_prompt)
print(llama3_inference_result)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


1978년에 최초로 미국과 일본의 방위협력지침이 만들어졌습니다.


In [ ]:
# 대표 LLM별 번역 추론 성능 비교
ground_truth_text = extracted_data[1]['answer']
llama_text = llama3_inference_result
gpt4_text = "최초로 미국과 일본의 방위협력지침이 만들어진 시기는 1978년입니다."
gpt35_text = "미국과 일본의 최초 방위협력지침은 1978년에 마련되었어요."
clova_text = "최초로 미국과 일본의 방위협력지침이 만들어진 것은 1978년입니다. 이후 1997년에 개정되었고, 2015년에 '신 미·일방위협력지침'에 합의하여 미·일 동맹의 군사적 활동 공간을 전 세계로 확장하였습니다."
gemini_text = "최초의 미국-일본 방위협력지침은 1978년에 만들어졌습니다."

# Call the function to display the example texts
print_text_comparison(ground_truth_text, llama_text, gpt4_text, gpt35_text, clova_text, gemini_text)

Ground Truth:
1978년

Llama:
1978년에 최초로 미국과 일본의 방위협력지침이 만들어졌습니다.

GPT-4:
최초로 미국과 일본의 방위협력지침이 만들어진 시기는 1978년입니다.

GPT-3.5:
미국과 일본의 최초 방위협력지침은 1978년에 마련되었어요.

Clova:
최초로 미국과 일본의 방위협력지침이 만들어진 것은 1978년입니다. 이후 1997년에 개정되었고, 2015년에 '신 미·일방위협력지침'에 합의하여 미·일 동맹의 군사적 활동 공간을 전 세계로 확장하였습니다.

Gemini:
최초의 미국-일본 방위협력지침은 1978년에 만들어졌습니다.


In [ ]:
extracted_data[2]

{'context': '한국의 경우에도 한국 정부의 동의 없이도 한반도 유사시 일본의 군사적 개입이 가능할 수도 있다는 해석이 제기되면서 이에 대한 비판적인 입장이 부각되고 있다. 특히 이번 신 미·일방위협력지침은 아베 총리 집권이후 진행되고 있는 일본의 우경화행보와 맞물려 있기 때문에 매우 우려스럽지 않을 수 없다.\n그러나 상황에 따라 힘의 논리가 지배하는 현실 국제질서를 감안할 때 신 미·일방위협력지침이 한반도 평화질서에 어떻게 영향을 미치게 될 지에 대해서 선험적으로 속단하기 어려운 측면도 있다는 일부의 주장도 무시할 수는 없다.\n이 글에서는 미·일방위협력지침의 제정 및 개정, 그리고 ‘신 미·일방위협력지침’의주요 내용을 개관한다. 그리고 이를 둘러싼 주요 쟁점을 정리하고, 이를 토대로 한국의 대응과제를 제시하고자 한다.',
 'question': '한반도에 일본의 군사적 개입 가능성을 열어두어 논란이 제기된 것은 뭐야',
 'answer': '신 미·일방위협력지침'}

In [ ]:
질문_prompt = format_context_question(extracted_data[2])
질문_prompt

'Context: 한국의 경우에도 한국 정부의 동의 없이도 한반도 유사시 일본의 군사적 개입이 가능할 수도 있다는 해석이 제기되면서 이에 대한 비판적인 입장이 부각되고 있다. 특히 이번 신 미·일방위협력지침은 아베 총리 집권이후 진행되고 있는 일본의 우경화행보와 맞물려 있기 때문에 매우 우려스럽지 않을 수 없다.\n그러나 상황에 따라 힘의 논리가 지배하는 현실 국제질서를 감안할 때 신 미·일방위협력지침이 한반도 평화질서에 어떻게 영향을 미치게 될 지에 대해서 선험적으로 속단하기 어려운 측면도 있다는 일부의 주장도 무시할 수는 없다.\n이 글에서는 미·일방위협력지침의 제정 및 개정, 그리고 ‘신 미·일방위협력지침’의주요 내용을 개관한다. 그리고 이를 둘러싼 주요 쟁점을 정리하고, 이를 토대로 한국의 대응과제를 제시하고자 한다.\nQuestion: 한반도에 일본의 군사적 개입 가능성을 열어두어 논란이 제기된 것은 뭐야'

In [ ]:
llama3_inference_result = generate_response(system_message="너는 주어진 Context를 토대로 Question에 답변하는 챗봇이야. Question에 대한 답변을 한국어로 답해줘.",
                             user_message=질문_prompt)
print(llama3_inference_result)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


한반도에 일본의 군사적 개입 가능성을 열어두어 논란이 제기된 것은 신 미·일방위협력지침입니다. 이 지침은 한국 정부의 동의 없이도 한반도에 일본의 군사적 개입이 가능할 수 있다는 해석이 제기되면서 논란이 제기된 것입니다.


In [ ]:
# 대표 LLM별 번역 추론 성능 비교
ground_truth_text = extracted_data[2]['answer']
llama_text = llama3_inference_result
gpt4_text = """한반도에 일본의 군사적 개입 가능성을 열어두어 논란이 제기된 것은 "신 미·일방위협력지침"입니다. 이 지침은 아베 총리 집권 이후 일본의 우경화 정책과 연결되어 있으며, 한국 정부의 동의 없이도 일본의 군사적 개입이 가능할 수 있다는 해석이 나오면서 비판적인 입장이 부각되었습니다. 이러한 지침이 한반도의 평화 질서에 어떤 영향을 미칠지에 대한 논의는 여전히 진행 중이며, 국제 질서의 현실을 감안한 다양한 의견이 존재합니다."""
gpt35_text = "한반도에 일본의 군사적 개입 가능성이 제기된 것은, 한국 정부의 동의 없이도 한반도에 대한 군사적 개입이 가능할 수 있다는 해석이 제기되었기 때문입니다. 특히 신 미·일방위협력지침이 이러한 가능성을 열어둘 수 있다는 우려가 있습니다."
clova_text = "최초로 미국과 일본의 방위협력지침이 만들어진 것은 1978년입니다. 이후 1997년에 개정되었고, 2015년에 '신 미·일방위협력지침'에 합의하여 미·일 동맹의 군사적 활동 공간을 전 세계로 확장하였습니다."
gemini_text = """한반도에 일본의 군사적 개입 가능성을 열어두어 논란이 제기된 이유는 다음과 같습니다.
1. 한국 정부의 동의 없이도 가능한 개입:

신 미일 방위협력지침은 한반도 유사시 일본의 군사적 개입을 명시적으로 규정하지는 않지만, '지역 공동의 안보 위협'에 대한 협력을 강조하면서 이를 해석에 따라 일본의 개입 가능성으로 연결할 수 있다는 우려가 제기되고 있습니다.
특히, '지역 공동의 안보 위협'이라는 개념이 명확하게 정의되지 않아 상황에 따라 일본이 자의적으로 해석하여 개입할 수 있다는 우려가 있습니다.
또한, 한반도 유사 발생 시 한국 정부의 동의 없이도 미국과의 협력 하에 일본이 개입할 수 있다는 해석도 가능합니다.
2. 아베 총리의 우경화 행보와 연관성:

아베 신조 일본 총리는 집권 이후 역사 인식 개선, 헌법 개정 등 우경화 행보를 지속해 왔습니다.
이러한 행보는 과거 일본의 식민지 침략과 범죄를 미화하고, 군사력을 강화하려는 의도로 해석될 수 있습니다.
따라서 신 미일 방위협력지침이 아베 총리의 우경화 행보와 맞물려 일본의 군사적 야욕을 실현하는 데 악용될 수 있다는 우려가 제기되고 있습니다.
3. 한반도 평화 질서에 미치는 영향:

일본의 군사적 개입 가능성은 한반도의 긴장을 고조시키고, 남북 대화와 협력을 저해할 수 있습니다.
또한, 동북아 지역의 군비 경쟁을 심화시키고, 새로운 안보 위협을 야기할 수 있다는 우려도 있습니다.
특히, 일본이 과거 식민지 지배 경험을 바탕으로 한반도에 영향력을 행사하려는 의도를 드러낼 경우, 이는 한반도 주민들의 반발과 갈등을 야기할 수 있습니다.
4. 기타 우려 사항:

일본의 군사 참여는 한반도 내 주민들의 안전을 위협할 수 있습니다.
또한, 일본의 군사 작전으로 인한 민간인 피해 발생 가능성도 배제할 수 없습니다.
더욱이, 일본의 군사 개입은 한반도의 비핵화 노력에도 부정적인 영향을 미칠 수 있습니다.
이처럼 신 미일 방위협력지침에 담긴 일본의 군사적 개입 가능성은 다양한 우려를 불러일으키고 있으며, 한반도 평화와 안정에 심각한 위협을 초래할 수 있다는 지적이 많습니다. 따라서 한국 정부는 이에 대해 명확한 입장을 밝히고, 적극적인 외교 노력을 통해 일본의 군사적 개입을 막아야 할 것입니다. 또한, 국민들의 안보 인식을 제고하고, 평화를 위한 노력을 지속해야 합니다.
"""

# Call the function to display the example texts
print_text_comparison(ground_truth_text, llama_text, gpt4_text, gpt35_text, clova_text, gemini_text)

Ground Truth:
신 미·일방위협력지침

Llama:
한반도에 일본의 군사적 개입 가능성을 열어두어 논란이 제기된 것은 신 미·일방위협력지침입니다. 이 지침은 한국 정부의 동의 없이도 한반도에 일본의 군사적 개입이 가능할 수 있다는 해석이 제기되면서 논란이 제기된 것입니다.

GPT-4:
한반도에 일본의 군사적 개입 가능성을 열어두어 논란이 제기된 것은 "신 미·일방위협력지침"입니다. 이 지침은 아베 총리 집권 이후 일본의 우경화 정책과 연결되어 있으며, 한국 정부의 동의 없이도 일본의 군사적 개입이 가능할 수 있다는 해석이 나오면서 비판적인 입장이 부각되었습니다. 이러한 지침이 한반도의 평화 질서에 어떤 영향을 미칠지에 대한 논의는 여전히 진행 중이며, 국제 질서의 현실을 감안한 다양한 의견이 존재합니다.

GPT-3.5:
한반도에 일본의 군사적 개입 가능성이 제기된 것은, 한국 정부의 동의 없이도 한반도에 대한 군사적 개입이 가능할 수 있다는 해석이 제기되었기 때문입니다. 특히 신 미·일방위협력지침이 이러한 가능성을 열어둘 수 있다는 우려가 있습니다.

Clova:
최초로 미국과 일본의 방위협력지침이 만들어진 것은 1978년입니다. 이후 1997년에 개정되었고, 2015년에 '신 미·일방위협력지침'에 합의하여 미·일 동맹의 군사적 활동 공간을 전 세계로 확장하였습니다.

Gemini:
한반도에 일본의 군사적 개입 가능성을 열어두어 논란이 제기된 이유는 다음과 같습니다.
1. 한국 정부의 동의 없이도 가능한 개입:

신 미일 방위협력지침은 한반도 유사시 일본의 군사적 개입을 명시적으로 규정하지는 않지만, '지역 공동의 안보 위협'에 대한 협력을 강조하면서 이를 해석에 따라 일본의 개입 가능성으로 연결할 수 있다는 우려가 제기되고 있습니다.
특히, '지역 공동의 안보 위협'이라는 개념이 명확하게 정의되지 않아 상황에 따라 일본이 자의적으로 해석하여 개입할 수 있다는 우려가 있습니다.
또한, 한반도 유사 발생 시 한국 정부의 동의 없이도 미국과의 협력 하

In [ ]:
extracted_data[3]

{'context': '한국의 경우에도 한국 정부의 동의 없이도 한반도 유사시 일본의 군사적 개입이 가능할 수도 있다는 해석이 제기되면서 이에 대한 비판적인 입장이 부각되고 있다. 특히 이번 신 미·일방위협력지침은 아베 총리 집권이후 진행되고 있는 일본의 우경화행보와 맞물려 있기 때문에 매우 우려스럽지 않을 수 없다.\n그러나 상황에 따라 힘의 논리가 지배하는 현실 국제질서를 감안할 때 신 미·일방위협력지침이 한반도 평화질서에 어떻게 영향을 미치게 될 지에 대해서 선험적으로 속단하기 어려운 측면도 있다는 일부의 주장도 무시할 수는 없다.\n이 글에서는 미·일방위협력지침의 제정 및 개정, 그리고 ‘신 미·일방위협력지침’의주요 내용을 개관한다. 그리고 이를 둘러싼 주요 쟁점을 정리하고, 이를 토대로 한국의 대응과제를 제시하고자 한다.',
 'question': '일본이 한반도에 군사력을 행사할 수도 있게 해서 문제가 된 건 뭐야',
 'answer': '신 미·일방위협력지침'}

In [ ]:
질문_prompt = format_context_question(extracted_data[3])
질문_prompt

'Context: 한국의 경우에도 한국 정부의 동의 없이도 한반도 유사시 일본의 군사적 개입이 가능할 수도 있다는 해석이 제기되면서 이에 대한 비판적인 입장이 부각되고 있다. 특히 이번 신 미·일방위협력지침은 아베 총리 집권이후 진행되고 있는 일본의 우경화행보와 맞물려 있기 때문에 매우 우려스럽지 않을 수 없다.\n그러나 상황에 따라 힘의 논리가 지배하는 현실 국제질서를 감안할 때 신 미·일방위협력지침이 한반도 평화질서에 어떻게 영향을 미치게 될 지에 대해서 선험적으로 속단하기 어려운 측면도 있다는 일부의 주장도 무시할 수는 없다.\n이 글에서는 미·일방위협력지침의 제정 및 개정, 그리고 ‘신 미·일방위협력지침’의주요 내용을 개관한다. 그리고 이를 둘러싼 주요 쟁점을 정리하고, 이를 토대로 한국의 대응과제를 제시하고자 한다.\nQuestion: 일본이 한반도에 군사력을 행사할 수도 있게 해서 문제가 된 건 뭐야'

In [ ]:
llama3_inference_result = generate_response(system_message="너는 주어진 Context를 토대로 Question에 답변하는 챗봇이야. Question에 대한 답변을 한국어로 답해줘.",
                             user_message=질문_prompt)
print(llama3_inference_result)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


일본이 한반도에 군사력을 행사할 수 있게 해주는 신 미·일방위협력지침이 문제가 된 것입니다. 이 지침은 한국 정부의 동의 없이도 일본의 군사적 개입을 가능하게 할 수 있습니다.


In [ ]:
# 대표 LLM별 번역 추론 성능 비교
ground_truth_text = extracted_data[3]['answer']
llama_text = llama3_inference_result
gpt4_text = "일본이 한반도에서 군사력을 행사할 수 있게 한 주된 문제는 한국 정부의 동의 없이도 일본의 군사적 개입이 가능할 수 있다는 점입니다. 이는 신 미·일방위협력지침에 따라 제기된 해석이며, 특히 이 지침이 일본의 우경화 행보와 연계되어 있다는 점에서 더욱 우려를 낳고 있습니다. 이러한 배경은 한반도의 안보 상황에 대한 주권적 결정권을 침해할 수 있으며, 지역 안정성에 대한 불확실성을 증가시킬 수 있습니다."
gpt35_text = "한반도에 군사력을 행사할 수 있는 가능성은 한국 정부 내외에서 큰 문제로 여겨지고 있어요. 특히 한국 정부의 동의 없이도 일본의 군사적 개입이 가능하다는 해석이 나오면서 비판적인 시각이 부각되고 있어요. 이는 아베 총리 집권 이후 진행되는 일본의 우국화 행보와 관련돼 있는데요, 이로 인해 우려가 커지고 있습니다."
clova_text = """일본이 한반도에 군사력을 행사할 수도 있게 해서 문제가 된 것은 다음과 같습니다.

1.한반도의 평화와 안정 저해: 일본의 군사적 개입은 한반도의 평화와 안정을 저해할 수 있습니다. 한반도 유사시 일본의 군사적 개입은 북한의 도발을 자극할 수 있으며, 이는 한반도의 긴장을 고조시킬 수 있습니다.

2.한국의 주권 침해: 일본의 군사적 개입은 한국의 주권을 침해할 수 있습니다. 한국 정부의 동의 없이 일본이 한반도에 군사력을 행사하는 것은 국제법상 문제가 될 수 있으며, 이는 한국의 주권을 침해하는 것으로 간주될 수 있습니다.

3.일본의 우경화 행보와 연관: 이번 신 미·일방위협력지침은 아베 총리 집권 이후 진행되고 있는 일본의 우경화 행보와 맞물려 있기 때문에 매우 우려스럽지 않을 수 없습니다. 일본의 군사적 개입은 일본의 군사 대국화를 촉진할 수 있으며, 이는 동아시아의 평화와 안정을 저해할 수 있습니다.

4.선험적으로 속단하기 어려운 측면: 상황에 따라 힘의 논리가 지배하는 현실 국제질서를 감안할 때 신 미·일방위협력지침이 한반도 평화질서에 어떻게 영향을 미치게 될 지에 대해서 선험적으로 속단하기 어려운 측면도 있습니다.

이러한 문제들은 한국의 안보와 국익에 직접적인 영향을 미칠 수 있으므로, 한국 정부는 일본의 군사적 개입에 대한 대응책을 마련해야 합니다.
"""
gemini_text = """신 미일 방위협력지침에서 일본의 한반도 군사 개입 가능성이 문제가 된 이유는 다음과 같습니다.
1. 한국 정부의 동의 없이 일본의 군사 개입 가능성:

문제점: 신 지침은 한반도 유사 발생 시 일본이 한국 정부의 동의 없이도 군사력을 행사할 수 있도록 해석될 수 있습니다. 이는 한국의 방위권 침해와 주권 침해 우려를 야기합니다.
근거:
지침 내용: 신 지침은 "지역 중요사태" 발생 시 미·일 양국이 협력하여 대응하도록 규정하고 있습니다. 여기에는 "한반도 유사"도 포함됩니다. 또한, "지역 중요사태"에 대한 명확한 정의가 없어 상황에 따라 일본이 자의적으로 해석하여 개입할 가능성이 있습니다.
역사적 경험: 과거 일본의 식민지 지배와 침략 역사를 고려할 때, 한국 국민들은 일본의 군사 개입에 대해 강한 우려를 가지고 있습니다.
2. 아베 총리 집권 이후 일본의 우경화 행보:

문제점: 아베 총리 집권 이후 일본은 과거 식민지 침략을 미화하고 군사력을 강화하는 등 우경화 행보를 보이고 있습니다. 이러한 일본의 행보는 한반도의 평화와 안정을 위협하는 요소로 작용하고 있습니다.
근거:
아베 정권의 주요 정책:
역사인식 개선: 아베 정권은 과거 식민지 침략을 미화하고 일본군의 역사를 왜곡하는 발언과 행동을 반복했습니다. 또한, '위안부' 문제 해결에도 소극적인 태도를 보이고 있습니다.
군사력 강화: 아베 정권은 집단적 자위권 행사容認, 방위비 대폭 증액, 첨단 무기 개발 등을 통해 군사력을 강화하고 있습니다. 또한, 해상 경비 및 영토 분쟁에서 강경한 입장을 취하고 있습니다.
3. 한반도 평화 질서에 대한 영향:

문제점: 일본의 한반도 군사 개입 가능성은 한반도의 긴장을 고조시키고, 남북 대화와 협력을 저해할 수 있습니다. 또한, 동북아 지역의 불안정을 야기할 수 있습니다.
근거:
한반도 긴장 고조: 일본의 군사 개입 가능성은 남북간의 군사적 대치를 심화시키고,偶発적인 군사적 충돌 위험을 높일 수 있습니다. 이는 한반도의 평화와 안정에 심각한 위협이 될 것입니다.
남북 대화 및 협력 저해: 일본의 군사 개입 가능성은 남북간의 불신을 심화시키고, 대화와 협력을 위한 분위기를 악화시킬 수 있습니다. 이는 한반도 평화 통일의 과정을 저해하는 요소가 될 것입니다.
동북아 불안정: 일본의 군사 개입 가능성은 동북아 지역의 군사적 균형을 깨뜨리고, 새로운 군사적 갈등을 야기할 수 있습니다. 이는 동북아 전체의 평화와 안정에 위협이 될 것입니다.
결론적으로, 일본이 한반도에 군사력을 행사할 수도 있게 해서 문제가 된 것은 한국의 방위권 침해, 역사적 경험, 아베 정권의 우경화 행보, 한반도 평화 질서에 대한 부정적 영향 등의 이유 때문입니다.
"""

# Call the function to display the example texts
print_text_comparison(ground_truth_text, llama_text, gpt4_text, gpt35_text, clova_text, gemini_text)

Ground Truth:
신 미·일방위협력지침

Llama:
일본이 한반도에 군사력을 행사할 수 있게 해주는 신 미·일방위협력지침이 문제가 된 것입니다. 이 지침은 한국 정부의 동의 없이도 일본의 군사적 개입을 가능하게 할 수 있습니다.

GPT-4:
일본이 한반도에서 군사력을 행사할 수 있게 한 주된 문제는 한국 정부의 동의 없이도 일본의 군사적 개입이 가능할 수 있다는 점입니다. 이는 신 미·일방위협력지침에 따라 제기된 해석이며, 특히 이 지침이 일본의 우경화 행보와 연계되어 있다는 점에서 더욱 우려를 낳고 있습니다. 이러한 배경은 한반도의 안보 상황에 대한 주권적 결정권을 침해할 수 있으며, 지역 안정성에 대한 불확실성을 증가시킬 수 있습니다.

GPT-3.5:
한반도에 군사력을 행사할 수 있는 가능성은 한국 정부 내외에서 큰 문제로 여겨지고 있어요. 특히 한국 정부의 동의 없이도 일본의 군사적 개입이 가능하다는 해석이 나오면서 비판적인 시각이 부각되고 있어요. 이는 아베 총리 집권 이후 진행되는 일본의 우국화 행보와 관련돼 있는데요, 이로 인해 우려가 커지고 있습니다.

Clova:
일본이 한반도에 군사력을 행사할 수도 있게 해서 문제가 된 것은 다음과 같습니다.

1.한반도의 평화와 안정 저해: 일본의 군사적 개입은 한반도의 평화와 안정을 저해할 수 있습니다. 한반도 유사시 일본의 군사적 개입은 북한의 도발을 자극할 수 있으며, 이는 한반도의 긴장을 고조시킬 수 있습니다.

2.한국의 주권 침해: 일본의 군사적 개입은 한국의 주권을 침해할 수 있습니다. 한국 정부의 동의 없이 일본이 한반도에 군사력을 행사하는 것은 국제법상 문제가 될 수 있으며, 이는 한국의 주권을 침해하는 것으로 간주될 수 있습니다.

3.일본의 우경화 행보와 연관: 이번 신 미·일방위협력지침은 아베 총리 집권 이후 진행되고 있는 일본의 우경화 행보와 맞물려 있기 때문에 매우 우려스럽지 않을 수 없습니다. 일본의 군사적 개입은 일본의 군사 대국화를 촉진할 수 있으며, 이는 동아시아의

In [ ]:
extracted_data[4]

{'context': '2. 미·일 동맹과 미·일방위협력지침\n현재의 미·일 동맹은 1960년 1월에 체결된 ‘미·일 상호협력 및 안전보장조약(미·일 신안보조약)’을 토대로 한다.\n이를 체계적으로 실행하기 위해서 1976년과 1978년에 각각 ‘미·일 안전보장협의위원회’와 ‘미·일방위협력지침’을 마련하였다.\n이후 미·일 동맹의 안보 영역이 일본영토에서 일본 주변지역으로 확대되면서 1997년에 ‘미·일방위협력지침’을 개정하였다. 그리고 최근 미일 동맹의 안보영역이 전 세계로 확대되면서 기존 미·일방위협력지침을 개정한 ‘신 미·일방위협력지침’을 마련하게 된 것이다.',
 'question': '미국과 일본이 맺은 동맹의 바탕이 되는 협약은 뭐야',
 'answer': '미·일 상호협력 및 안전보장조약'}

In [ ]:
질문_prompt = format_context_question(extracted_data[4])
질문_prompt

'Context: 2. 미·일 동맹과 미·일방위협력지침\n현재의 미·일 동맹은 1960년 1월에 체결된 ‘미·일 상호협력 및 안전보장조약(미·일 신안보조약)’을 토대로 한다.\n이를 체계적으로 실행하기 위해서 1976년과 1978년에 각각 ‘미·일 안전보장협의위원회’와 ‘미·일방위협력지침’을 마련하였다.\n이후 미·일 동맹의 안보 영역이 일본영토에서 일본 주변지역으로 확대되면서 1997년에 ‘미·일방위협력지침’을 개정하였다. 그리고 최근 미일 동맹의 안보영역이 전 세계로 확대되면서 기존 미·일방위협력지침을 개정한 ‘신 미·일방위협력지침’을 마련하게 된 것이다.\nQuestion: 미국과 일본이 맺은 동맹의 바탕이 되는 협약은 뭐야'

In [ ]:
llama3_inference_result = generate_response(system_message="너는 주어진 Context를 토대로 Question에 답변하는 챗봇이야. Question에 대한 답변을 한국어로 답해줘.",
                             user_message=질문_prompt)
print(llama3_inference_result)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


미·일 동맹의 바탕이 되는 협약은 '미·일 상호협력 및 안전보장조약(미·일 신안보조약)'입니다.


In [ ]:
# 대표 LLM별 번역 추론 성능 비교
ground_truth_text = extracted_data[4]['answer']
llama_text = llama3_inference_result
gpt4_text = "미국과 일본이 맺은 동맹의 바탕이 되는 협약은 ‘미·일 상호협력 및 안전보장조약(미·일 신안보조약)’입니다. 이 조약은 1960년 1월에 체결되었습니다."
gpt35_text = "미국과 일본이 맺은 동맹의 바탕이 되는 협약은 '미·일 상호협력 및 안전보장조약(미·일 신안보조약)'이다."
clova_text = "미국과 일본이 맺은 동맹의 바탕이 되는 협약은 1960년 1월에 체결된 ‘미·일 상호협력 및 안전보장조약(미·일 신안보조약)’입니다."
gemini_text = """미국과 일본이 맺은 동맹의 바탕이 되는 협약은 **미·일 상호협력 및 안전보장조약(미·일 신안보조약)**입니다.
1960년 1월에 체결된 이 조약은 다음과 같은 내용을 담고 있습니다.

양국은 공격에 대비하여 공동으로 방위한다.
일본의 안전은 미국의 안전에 필수적이다.
양국은 긴밀한 정치적, 경제적 협력을 한다.
이 조약은 미·일 동맹의 기반으로 작용하며, 양국의 안보 협력을 강화하는 데 중요한 역할을 하고 있습니다.

참고:

미·일 안전보장협의위원회: 미·일 양국이 안보 문제를 논의하는 위원회
미·일방위협력지침: 미·일 동맹의 구체적인 운영 방식을 정한 지침
"""

# Call the function to display the example texts
print_text_comparison(ground_truth_text, llama_text, gpt4_text, gpt35_text, clova_text, gemini_text)

Ground Truth:
미·일 상호협력 및 안전보장조약

Llama:
미·일 동맹의 바탕이 되는 협약은 '미·일 상호협력 및 안전보장조약(미·일 신안보조약)'입니다.

GPT-4:
미국과 일본이 맺은 동맹의 바탕이 되는 협약은 ‘미·일 상호협력 및 안전보장조약(미·일 신안보조약)’입니다. 이 조약은 1960년 1월에 체결되었습니다.

GPT-3.5:
미국과 일본이 맺은 동맹의 바탕이 되는 협약은 '미·일 상호협력 및 안전보장조약(미·일 신안보조약)'이다.

Clova:
미국과 일본이 맺은 동맹의 바탕이 되는 협약은 1960년 1월에 체결된 ‘미·일 상호협력 및 안전보장조약(미·일 신안보조약)’입니다.

Gemini:
미국과 일본이 맺은 동맹의 바탕이 되는 협약은 **미·일 상호협력 및 안전보장조약(미·일 신안보조약)**입니다.
1960년 1월에 체결된 이 조약은 다음과 같은 내용을 담고 있습니다.

양국은 공격에 대비하여 공동으로 방위한다.
일본의 안전은 미국의 안전에 필수적이다.
양국은 긴밀한 정치적, 경제적 협력을 한다.
이 조약은 미·일 동맹의 기반으로 작용하며, 양국의 안보 협력을 강화하는 데 중요한 역할을 하고 있습니다.

참고:

미·일 안전보장협의위원회: 미·일 양국이 안보 문제를 논의하는 위원회
미·일방위협력지침: 미·일 동맹의 구체적인 운영 방식을 정한 지침



In [ ]:
extracted_data[5]

{'context': '2. 미·일 동맹과 미·일방위협력지침\n현재의 미·일 동맹은 1960년 1월에 체결된 ‘미·일 상호협력 및 안전보장조약(미·일 신안보조약)’을 토대로 한다.\n이를 체계적으로 실행하기 위해서 1976년과 1978년에 각각 ‘미·일 안전보장협의위원회’와 ‘미·일방위협력지침’을 마련하였다.\n이후 미·일 동맹의 안보 영역이 일본영토에서 일본 주변지역으로 확대되면서 1997년에 ‘미·일방위협력지침’을 개정하였다. 그리고 최근 미일 동맹의 안보영역이 전 세계로 확대되면서 기존 미·일방위협력지침을 개정한 ‘신 미·일방위협력지침’을 마련하게 된 것이다.',
 'question': '무슨 협약을 근간으로 미국과 일본 사이의 동맹이 이루어졌어',
 'answer': '미·일 상호협력 및 안전보장조약'}

In [ ]:
질문_prompt = format_context_question(extracted_data[5])
질문_prompt

'Context: 2. 미·일 동맹과 미·일방위협력지침\n현재의 미·일 동맹은 1960년 1월에 체결된 ‘미·일 상호협력 및 안전보장조약(미·일 신안보조약)’을 토대로 한다.\n이를 체계적으로 실행하기 위해서 1976년과 1978년에 각각 ‘미·일 안전보장협의위원회’와 ‘미·일방위협력지침’을 마련하였다.\n이후 미·일 동맹의 안보 영역이 일본영토에서 일본 주변지역으로 확대되면서 1997년에 ‘미·일방위협력지침’을 개정하였다. 그리고 최근 미일 동맹의 안보영역이 전 세계로 확대되면서 기존 미·일방위협력지침을 개정한 ‘신 미·일방위협력지침’을 마련하게 된 것이다.\nQuestion: 무슨 협약을 근간으로 미국과 일본 사이의 동맹이 이루어졌어'

In [ ]:
llama3_inference_result = generate_response(system_message="너는 주어진 Context를 토대로 Question에 답변하는 챗봇이야. Question에 대한 답변을 한국어로 답해줘.",
                             user_message=질문_prompt)
print(llama3_inference_result)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


미국과 일본 사이의 동맹은 1960년 1월에 체결된 '미·일 상호협력 및 안전보장조약(미·일 신안보조약)'을 근간으로 이루어졌습니다.


In [ ]:
# 대표 LLM별 번역 추론 성능 비교
ground_truth_text = extracted_data[5]['answer']
llama_text = llama3_inference_result
gpt4_text = "미국과 일본 사이의 동맹은 1960년 1월에 체결된 '미·일 상호협력 및 안전보장조약(미·일 신안보조약)'을 근간으로 이루어졌습니다."
gpt35_text = "미국과 일본 사이의 동맹은 1960년 1월에 체결된 '미·일 상호협력 및 안전보장조약(미·일 신안보조약)'을 근간으로 합니다."
clova_text = "미국과 일본 사이의 동맹은 1960년 1월에 체결된 ‘미·일 상호협력 및 안전보장조약(미·일 신안보조약)’을 근간으로 이루어졌습니다."
gemini_text = """**미국과 일본 사이의 동맹은 1960년 1월 19일에 체결된 **미일 상호협력 및 안전보장 조약(미일 신안보조약)을 근간으로 이루어졌습니다.

이 조약은 제2차 세계 대전 이후 일본의 안전을 보장하기 위해 미국이 일본을 방위할 의무를 규정하고 있으며, 일본은 미국과 협력하여 공동의 안보를 유지할 것을 의무지고 있습니다.

미일 신안보조약은 미일 안전보장 협의위원회와 미일 방위협력지침이라는 두 가지 보조적인 합의를 통해 구체화되고 있습니다.

미일 안전보장 협의위원회: 양국 외교·국방장관급으로 구성된 위원회로서, 동맹관련 주요 사안을 협의하고 결정하는 역할을 합니다.
미일 방위협력지침: 양국의 군사력을 어떻게 통합하여 운영할지를 구체적으로 규정한 지침입니다. 1997년과 2017년에 개정되었으며, 최근 개정된 신 미일 방위협력지침에서는 우주, 사이버, 전자공격 등 새로운 안보 위협에 대응하기 위한 협력 강화를 중점적으로 다루고 있습니다.
미일 동맹은 동아시아 지역의 평화와 안정에 중요한 역할을 하고 있으며, 최근에는 중국의 군사적 성장에 대응하기 위한 양국 간 협력이 더욱 강화되고 있습니다.
"""

# Call the function to display the example texts
print_text_comparison(ground_truth_text, llama_text, gpt4_text, gpt35_text, clova_text, gemini_text)

Ground Truth:
미·일 상호협력 및 안전보장조약

Llama:
미국과 일본 사이의 동맹은 1960년 1월에 체결된 '미·일 상호협력 및 안전보장조약(미·일 신안보조약)'을 근간으로 이루어졌습니다.

GPT-4:
미국과 일본 사이의 동맹은 1960년 1월에 체결된 '미·일 상호협력 및 안전보장조약(미·일 신안보조약)'을 근간으로 이루어졌습니다.

GPT-3.5:
미국과 일본 사이의 동맹은 1960년 1월에 체결된 '미·일 상호협력 및 안전보장조약(미·일 신안보조약)'을 근간으로 합니다.

Clova:
미국과 일본 사이의 동맹은 1960년 1월에 체결된 ‘미·일 상호협력 및 안전보장조약(미·일 신안보조약)’을 근간으로 이루어졌습니다.

Gemini:
**미국과 일본 사이의 동맹은 1960년 1월 19일에 체결된 **미일 상호협력 및 안전보장 조약(미일 신안보조약)을 근간으로 이루어졌습니다.

이 조약은 제2차 세계 대전 이후 일본의 안전을 보장하기 위해 미국이 일본을 방위할 의무를 규정하고 있으며, 일본은 미국과 협력하여 공동의 안보를 유지할 것을 의무지고 있습니다.

미일 신안보조약은 미일 안전보장 협의위원회와 미일 방위협력지침이라는 두 가지 보조적인 합의를 통해 구체화되고 있습니다.

미일 안전보장 협의위원회: 양국 외교·국방장관급으로 구성된 위원회로서, 동맹관련 주요 사안을 협의하고 결정하는 역할을 합니다.
미일 방위협력지침: 양국의 군사력을 어떻게 통합하여 운영할지를 구체적으로 규정한 지침입니다. 1997년과 2017년에 개정되었으며, 최근 개정된 신 미일 방위협력지침에서는 우주, 사이버, 전자공격 등 새로운 안보 위협에 대응하기 위한 협력 강화를 중점적으로 다루고 있습니다.
미일 동맹은 동아시아 지역의 평화와 안정에 중요한 역할을 하고 있으며, 최근에는 중국의 군사적 성장에 대응하기 위한 양국 간 협력이 더욱 강화되고 있습니다.



In [ ]:
extracted_data[6]

{'context': '3. 신 미·일방위협력지침의 주요내용\n신 미·일방위협력지침은 다음과 같은 측면에서 1997년의 미·일방위협력지침과 차이가 있다.\n첫째, 미·일 동맹의 활동범위가 기존의 방위협력지침에서 설정한 일본과 주변지역은 물론 아·태지역 및 전 세계로 확대되었다.\n둘째, 안보협력상황을 ‘평화시’, ‘일본의 평화와 안보에 대한 중대한 위협상황’, ‘일본이 군사적 외침을 받는 상황’, ‘일본이 아닌 미국과 다른 나라가 군사적 외침을 받는 상황 ’, ‘일본에서 대규모 재해가 발생한 상황’ 등 5가지로 확대하였다.\n셋째, 미·일 동맹 협력추진체계를 확대하고 통합성을 강화하였다. 이를 위해 미·일 동맹 협력을 범정부적 접근법이라고 명시하고 상설‘동맹협력메카니즘’(Alliance Coordination Mechanism)을 구축하기로 하였다.\n넷째, 평시의 안보협력 범주를 확대하였다. 이 전과 달리 정보와 감시활동은 물론 탐지(ISR: Intelligence, Suveillance, Reconnaissance) 활동까지 통합적으로 협력하기로 하였다. 또한 평시에 탄도미사일을 포함하는 공중방어와 해상안보까지 양자 간의 안보협력 범주에 포함하였다.',
 'question': '새로 만든 미·일방위협력지침에서 미·일 동맹 협력추진체계를 증대하기 위해 구축하려는 게 뭐야',
 'answer': '동맹협력메카니즘'}

In [ ]:
질문_prompt = format_context_question(extracted_data[6])
질문_prompt

'Context: 3. 신 미·일방위협력지침의 주요내용\n신 미·일방위협력지침은 다음과 같은 측면에서 1997년의 미·일방위협력지침과 차이가 있다.\n첫째, 미·일 동맹의 활동범위가 기존의 방위협력지침에서 설정한 일본과 주변지역은 물론 아·태지역 및 전 세계로 확대되었다.\n둘째, 안보협력상황을 ‘평화시’, ‘일본의 평화와 안보에 대한 중대한 위협상황’, ‘일본이 군사적 외침을 받는 상황’, ‘일본이 아닌 미국과 다른 나라가 군사적 외침을 받는 상황 ’, ‘일본에서 대규모 재해가 발생한 상황’ 등 5가지로 확대하였다.\n셋째, 미·일 동맹 협력추진체계를 확대하고 통합성을 강화하였다. 이를 위해 미·일 동맹 협력을 범정부적 접근법이라고 명시하고 상설‘동맹협력메카니즘’(Alliance Coordination Mechanism)을 구축하기로 하였다.\n넷째, 평시의 안보협력 범주를 확대하였다. 이 전과 달리 정보와 감시활동은 물론 탐지(ISR: Intelligence, Suveillance, Reconnaissance) 활동까지 통합적으로 협력하기로 하였다. 또한 평시에 탄도미사일을 포함하는 공중방어와 해상안보까지 양자 간의 안보협력 범주에 포함하였다.\nQuestion: 새로 만든 미·일방위협력지침에서 미·일 동맹 협력추진체계를 증대하기 위해 구축하려는 게 뭐야'

In [ ]:
llama3_inference_result = generate_response(system_message="너는 주어진 Context를 토대로 Question에 답변하는 챗봇이야. Question에 대한 답변을 한국어로 답해줘.",
                             user_message=질문_prompt)
print(llama3_inference_result)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


신 미·일방위협력지침에서 미·일 동맹 협력추진체계를 증대하기 위해 구축하려는 것은 '상설‘동맹협력메카니즘’(Alliance Coordination Mechanism)'입니다. 이 메카니즘은 미·일 동맹 협력을 범정부적 접근법으로 명시하고, 이를 통해 미·일 동맹의 통합성을 강화하고 협력추진체계를 확대하고자 하는 것입니다.


In [ ]:
# 대표 LLM별 번역 추론 성능 비교
ground_truth_text = extracted_data[6]['answer']
llama_text = llama3_inference_result
gpt4_text = "새로 만든 미·일방위협력지침에서 미·일 동맹 협력추진체계를 증대하기 위해 구축하려는 것은 상설 '동맹협력메카니즘' (Alliance Coordination Mechanism)이다. 이는 미국과 일본간의 동맹 협력을 범정부적 접근법으로 강화하고 통합성을 높이기 위한 조치로 명시되어 있다."
gpt35_text = "미·일 동맹 협력추진체계를 증대하기 위해 구축하려는 것은 상설 '동맹협력메카니즘'입니다."
clova_text = "새로 만든 미·일방위협력지침에서 미·일 동맹 협력추진체계를 증대하기 위해 구축하려는 것은 상설 '동맹협력메카니즘'(Alliance Coordination Mechanism)입니다."
gemini_text = """새로 만든 미·일 방위협력지침에서 미·일 동맹 협력추진체계를 증대하기 위해 구축하려는 것은 **상설 '동맹협력메카니즘'(Alliance Coordination Mechanism)**입니다.

이 메커니즘은 미·일 양국의 외교, 국방, 해양안보 등 각 부처의 고위 관료들로 구성된 합동기구로서, 다음과 같은 역할을 수행하게 됩니다.

**미·일 동맹 협력 정책 및 계획의 **협의 및 조율
**양국 군사력의 연합훈련 및 운영 협력
**정보 및 정찰 활동 공동 수행
**미·일 간의 군사기술 및 장비 호환성 향상 협력
**지역 및 국제 안보 문제에 대한 협의 협력
핵심 목표는 다음과 같습니다.

**미·일 동맹의 효율성 및 효과성 극대화
**양국 간의 **군사적 협력 강화
**아시아태평양 지역의 평화 및 안정 유지
동맹협력메카니즘은 다음과 같은 구성원으로 구성됩니다.

외교, 국방, 해양안보 각 부처의 고위 관료
합동참모본부 기획 참모
각 군 사령관
**필요에 따라 전문가
운영 방식은 다음과 같습니다.

**정기적인 회의 개최
**비상시 긴급 연락 체계 구축
**영구적인 사무실 운영
기대 효과는 다음과 같습니다.

**미·일 동맹의 강화
**아시아태평양 지역의 안보 환경 개선
**국제 사회의 평화 및 안정 에 기여
동맹협력메카니즘은 아직 구체적인 운영 방식이 결정되지 않은 부분도 있지만, 미·일 동맹 협력 강화에 중추적인 역할을 할 것으로 기대됩니다.
"""

# Call the function to display the example texts
print_text_comparison(ground_truth_text, llama_text, gpt4_text, gpt35_text, clova_text, gemini_text)

Ground Truth:
동맹협력메카니즘

Llama:
신 미·일방위협력지침에서 미·일 동맹 협력추진체계를 증대하기 위해 구축하려는 것은 '상설‘동맹협력메카니즘’(Alliance Coordination Mechanism)'입니다. 이 메카니즘은 미·일 동맹 협력을 범정부적 접근법으로 명시하고, 이를 통해 미·일 동맹의 통합성을 강화하고 협력추진체계를 확대하고자 하는 것입니다.

GPT-4:
새로 만든 미·일방위협력지침에서 미·일 동맹 협력추진체계를 증대하기 위해 구축하려는 것은 상설 '동맹협력메카니즘' (Alliance Coordination Mechanism)이다. 이는 미국과 일본간의 동맹 협력을 범정부적 접근법으로 강화하고 통합성을 높이기 위한 조치로 명시되어 있다.

GPT-3.5:
미·일 동맹 협력추진체계를 증대하기 위해 구축하려는 것은 상설 '동맹협력메카니즘'입니다.

Clova:
새로 만든 미·일방위협력지침에서 미·일 동맹 협력추진체계를 증대하기 위해 구축하려는 것은 상설 '동맹협력메카니즘'(Alliance Coordination Mechanism)입니다.

Gemini:
새로 만든 미·일 방위협력지침에서 미·일 동맹 협력추진체계를 증대하기 위해 구축하려는 것은 **상설 '동맹협력메카니즘'(Alliance Coordination Mechanism)**입니다.

이 메커니즘은 미·일 양국의 외교, 국방, 해양안보 등 각 부처의 고위 관료들로 구성된 합동기구로서, 다음과 같은 역할을 수행하게 됩니다.

**미·일 동맹 협력 정책 및 계획의 **협의 및 조율
**양국 군사력의 연합훈련 및 운영 협력
**정보 및 정찰 활동 공동 수행
**미·일 간의 군사기술 및 장비 호환성 향상 협력
**지역 및 국제 안보 문제에 대한 협의 협력
핵심 목표는 다음과 같습니다.

**미·일 동맹의 효율성 및 효과성 극대화
**양국 간의 **군사적 협력 강화
**아시아태평양 지역의 평화 및 안정 유지
동맹협력메카니즘은 다음과 같은 구성원으로 구성됩니다.

외교, 국방, 해양

In [ ]:
extracted_data[7]

{'context': '3. 신 미·일방위협력지침의 주요내용\n신 미·일방위협력지침은 다음과 같은 측면에서 1997년의 미·일방위협력지침과 차이가 있다.\n첫째, 미·일 동맹의 활동범위가 기존의 방위협력지침에서 설정한 일본과 주변지역은 물론 아·태지역 및 전 세계로 확대되었다.\n둘째, 안보협력상황을 ‘평화시’, ‘일본의 평화와 안보에 대한 중대한 위협상황’, ‘일본이 군사적 외침을 받는 상황’, ‘일본이 아닌 미국과 다른 나라가 군사적 외침을 받는 상황 ’, ‘일본에서 대규모 재해가 발생한 상황’ 등 5가지로 확대하였다.\n셋째, 미·일 동맹 협력추진체계를 확대하고 통합성을 강화하였다. 이를 위해 미·일 동맹 협력을 범정부적 접근법이라고 명시하고 상설‘동맹협력메카니즘’(Alliance Coordination Mechanism)을 구축하기로 하였다.\n넷째, 평시의 안보협력 범주를 확대하였다. 이 전과 달리 정보와 감시활동은 물론 탐지(ISR: Intelligence, Suveillance, Reconnaissance) 활동까지 통합적으로 협력하기로 하였다. 또한 평시에 탄도미사일을 포함하는 공중방어와 해상안보까지 양자 간의 안보협력 범주에 포함하였다.',
 'question': '새로운 미·일방위협력지침에 따르면 무엇을 통해 미국과 일본의 동맹 협력추진체계를 확대하려고 해',
 'answer': '동맹협력메카니즘'}

In [ ]:
질문_prompt = format_context_question(extracted_data[7])
질문_prompt

'Context: 3. 신 미·일방위협력지침의 주요내용\n신 미·일방위협력지침은 다음과 같은 측면에서 1997년의 미·일방위협력지침과 차이가 있다.\n첫째, 미·일 동맹의 활동범위가 기존의 방위협력지침에서 설정한 일본과 주변지역은 물론 아·태지역 및 전 세계로 확대되었다.\n둘째, 안보협력상황을 ‘평화시’, ‘일본의 평화와 안보에 대한 중대한 위협상황’, ‘일본이 군사적 외침을 받는 상황’, ‘일본이 아닌 미국과 다른 나라가 군사적 외침을 받는 상황 ’, ‘일본에서 대규모 재해가 발생한 상황’ 등 5가지로 확대하였다.\n셋째, 미·일 동맹 협력추진체계를 확대하고 통합성을 강화하였다. 이를 위해 미·일 동맹 협력을 범정부적 접근법이라고 명시하고 상설‘동맹협력메카니즘’(Alliance Coordination Mechanism)을 구축하기로 하였다.\n넷째, 평시의 안보협력 범주를 확대하였다. 이 전과 달리 정보와 감시활동은 물론 탐지(ISR: Intelligence, Suveillance, Reconnaissance) 활동까지 통합적으로 협력하기로 하였다. 또한 평시에 탄도미사일을 포함하는 공중방어와 해상안보까지 양자 간의 안보협력 범주에 포함하였다.\nQuestion: 새로운 미·일방위협력지침에 따르면 무엇을 통해 미국과 일본의 동맹 협력추진체계를 확대하려고 해'

In [ ]:
llama3_inference_result = generate_response(system_message="너는 주어진 Context를 토대로 Question에 답변하는 챗봇이야. Question에 대한 답변을 한국어로 답해줘.",
                             user_message=질문_prompt)
print(llama3_inference_result)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


신 미·일방위협력지침에 따르면, 미국과 일본의 동맹 협력추진체계를 확대하기 위해 상설 '동맹협력메카니즘'(Alliance Coordination Mechanism)을 구축하기로 하였다.


In [ ]:
# 대표 LLM별 번역 추론 성능 비교
ground_truth_text = extracted_data[7]['answer']
llama_text = llama3_inference_result
gpt4_text = "새로운 미·일방위협력지침에 따르면 미국과 일본은 '동맹협력메카니즘'(Alliance Coordination Mechanism)을 구축하여 동맹 협력추진체계를 확대하려고 합니다. 이 메카니즘은 범정부적 접근법을 기반으로 하며, 동맹 간의 협력 통합성을 강화하는 데 목적을 두고 있습니다."
gpt35_text = "새로운 미·일방위협력지침에 따르면 미국과 일본의 동맹 협력추진체계를 확대하기 위해 상설 '동맹협력메카니즘'을 구축하고, 미·일 동맹 협력을 범정부적으로 접근하는 방법을 명시했습니다."
clova_text = "새로운 미·일방위협력지침에 따르면, 미국과 일본의 동맹 협력추진체계를 확대하고 통합성을 강화하기 위해 '동맹협력메카니즘'(Alliance Coordination Mechanism)을 구축하기로 하였습니다."
gemini_text = """새로운 미·일방위협력지침에 따르면 미국과 일본은 다음과 같은 방식으로 동맹 협력 추진 체계를 확대하려고 합니다.

1. 범정부적 접근법 도입:

양국 정부의 모든 부처와 기관이 동맹 협력에 참여하도록 하여 협력의 범위와 심도를 확대합니다.
외교, 국방, 경제, 과학기술 등 모든 분야에서 상호 협력을 강화합니다.
2. 상설 '동맹협력메카니즘' 구축:

양국 정부 고위 관료로 구성된 '동맹협력메카니즘'을 설립하여 정기적인 협의와 협력을 진행합니다.
각 부처 및 기관 간의 정보 공유와 협력을 활성화하고, 정책을 조율합니다.
동맹 협력 관련 현안을 신속かつ 효율적으로 처리합니다.
3. 정보 공유 및 협력 강화:

정보와 감시 활동, 탐지 활동 등을 통합적으로 협력하여 상호 안보 위협에 대한 인지 능력을 강화합니다.
사이버 공격, 테러, 해양 안보 위협 등 다양한 분야에서 정보를 공유하고 협력하여 대응합니다.
우주, 전자전 등 신기술 분야의 정보 공유도 적극적으로 추진합니다.
4. 평시 안보 협력 확대:

탄도 미사일 방어, 해상 안보 등 평시에도 발생할 수 있는 안보 위협에 대비하여 협력합니다.
공동 군사 훈련, 연합 작전 등을 통해 상호 운영 능력을 강화합니다.
재난 발생 시, 인도주의적 지원 및 구조 활동에서도 협력합니다.
5. 아태지역 및 전 세계 협력:

동맹 협력 범위를 기존의 일본과 주변 지역에서 아태지역 및 전 세계로 확대합니다.
인도-태평양 지역의 자유로운 항해, 질서 유지, 경제 번영을 위해 협력합니다.
국제 사회의 평화와 안정에 기여하기 위해 공동으로 노력합니다.
이처럼 새로운 미·일방위협력지침은 다양한 분야에서의 협력을 강화하고, 동맹 협력 체계를 보다 체계적이고 효율적으로 구축하기 위한 구체적인 방안을 제시하고 있습니다. 이는 양국의 안보를 강화하고, 아태지역 및 전 세계의 평화와 안정에 기여하는 데 중요한 역할을 할 것으로 기대됩니다.
"""

# Call the function to display the example texts
print_text_comparison(ground_truth_text, llama_text, gpt4_text, gpt35_text, clova_text, gemini_text)

Ground Truth:
동맹협력메카니즘

Llama:
신 미·일방위협력지침에 따르면, 미국과 일본의 동맹 협력추진체계를 확대하기 위해 상설 '동맹협력메카니즘'(Alliance Coordination Mechanism)을 구축하기로 하였다.

GPT-4:
새로운 미·일방위협력지침에 따르면 미국과 일본은 '동맹협력메카니즘'(Alliance Coordination Mechanism)을 구축하여 동맹 협력추진체계를 확대하려고 합니다. 이 메카니즘은 범정부적 접근법을 기반으로 하며, 동맹 간의 협력 통합성을 강화하는 데 목적을 두고 있습니다.

GPT-3.5:
새로운 미·일방위협력지침에 따르면 미국과 일본의 동맹 협력추진체계를 확대하기 위해 상설 '동맹협력메카니즘'을 구축하고, 미·일 동맹 협력을 범정부적으로 접근하는 방법을 명시했습니다.

Clova:
새로운 미·일방위협력지침에 따르면, 미국과 일본의 동맹 협력추진체계를 확대하고 통합성을 강화하기 위해 '동맹협력메카니즘'(Alliance Coordination Mechanism)을 구축하기로 하였습니다.

Gemini:
새로운 미·일방위협력지침에 따르면 미국과 일본은 다음과 같은 방식으로 동맹 협력 추진 체계를 확대하려고 합니다.

1. 범정부적 접근법 도입:

양국 정부의 모든 부처와 기관이 동맹 협력에 참여하도록 하여 협력의 범위와 심도를 확대합니다.
외교, 국방, 경제, 과학기술 등 모든 분야에서 상호 협력을 강화합니다.
2. 상설 '동맹협력메카니즘' 구축:

양국 정부 고위 관료로 구성된 '동맹협력메카니즘'을 설립하여 정기적인 협의와 협력을 진행합니다.
각 부처 및 기관 간의 정보 공유와 협력을 활성화하고, 정책을 조율합니다.
동맹 협력 관련 현안을 신속かつ 효율적으로 처리합니다.
3. 정보 공유 및 협력 강화:

정보와 감시 활동, 탐지 활동 등을 통합적으로 협력하여 상호 안보 위협에 대한 인지 능력을 강화합니다.
사이버 공격, 테러, 해양 안보 위협 등 다양한 분야에서 정보를 공유하고 협력하여 대응합니다.

In [ ]:
extracted_data[8]

{'context': '4. 신 미·일방위협력지침의 주요쟁점\n(1) 한국의 외교적 고립화\n신 미·일방위협력지침에 대해 국내 일각에서는 한국의 외교적 고립화를 주장하고 있다. 이러한 주장은 지난 2015년 4월 22일 인도네시아 자카르타에서 개최된 반둥회의를 계기로 중·일 정상회담이 개최되는 등 중·일관계도 정상화되는 듯한 분위기가 형성되면서 더욱 부각되고 있는데, 이는 한국이 일본과의 갈등관계를 유지하면서도 실제적인 성과가 없다는 판단에 따른 것이다. 즉 외교전략의 부재로 인해 대 미국과 대 중국 및 대 일본관계에서 한국의 국익을 제대로 챙기지 못하고 있다는 주장이다.\n',
 'question': '한국의 외교적 고립에 대한 염려를 더 크게 한 중일 정상회담은 무엇으로 인해 성사됐어',
 'answer': '반둥회의'}

In [ ]:
질문_prompt = format_context_question(extracted_data[8])
질문_prompt

'Context: 4. 신 미·일방위협력지침의 주요쟁점\n(1) 한국의 외교적 고립화\n신 미·일방위협력지침에 대해 국내 일각에서는 한국의 외교적 고립화를 주장하고 있다. 이러한 주장은 지난 2015년 4월 22일 인도네시아 자카르타에서 개최된 반둥회의를 계기로 중·일 정상회담이 개최되는 등 중·일관계도 정상화되는 듯한 분위기가 형성되면서 더욱 부각되고 있는데, 이는 한국이 일본과의 갈등관계를 유지하면서도 실제적인 성과가 없다는 판단에 따른 것이다. 즉 외교전략의 부재로 인해 대 미국과 대 중국 및 대 일본관계에서 한국의 국익을 제대로 챙기지 못하고 있다는 주장이다.\n\nQuestion: 한국의 외교적 고립에 대한 염려를 더 크게 한 중일 정상회담은 무엇으로 인해 성사됐어'

In [ ]:
llama3_inference_result = generate_response(system_message="너는 주어진 Context를 토대로 Question에 답변하는 챗봇이야. Question에 대한 답변을 한국어로 답해줘.",
                             user_message=질문_prompt)
print(llama3_inference_result)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


중일 정상회담이 성사된 것은 2015년 4월 22일 인도네시아 자카르타에서 개최된 반둥회의를 계기로 발생한 중일관계의 정상화 분위기 때문입니다.


In [ ]:
# 대표 LLM별 번역 추론 성능 비교
ground_truth_text = extracted_data[8]['answer']
llama_text = llama3_inference_result
gpt4_text = "한국의 외교적 고립에 대한 염려를 더 크게 한 중일 정상회담은 2015년 4월 22일 인도네시아 자카르타에서 개최된 반둥회의를 계기로 성사되었습니다. 이 회의를 통해 중·일 간의 관계가 정상화되는 분위기가 형성되었고, 이는 한국이 일본과의 갈등을 유지하고 있음에도 불구하고 외교적으로 실질적인 성과를 보지 못하는 상황과 맞물려 한국의 외교적 고립화에 대한 우려를 증폭시켰습니다."
gpt35_text = "중일 정상회담은 2015년 4월 22일에 개최된 반둥회의를 계기로 이루어졌어요. 이 회의를 통해 중일 관계가 일부 정상화되는 분위기가 형성되면서 정상회담이 이루어졌습니다."
clova_text = "한국의 외교적 고립에 대한 염려를 더 크게 한 중일 정상회담은 2015년 4월 22일 인도네시아 자카르타에서 개최된 반둥회의를 계기로 성사되었습니다. 이 회담은 중·일관계가 정상화되는 듯한 분위기가 형성되면서 주목을 받았습니다."
gemini_text = """4. 신 미·일방위협력지침의 주요 쟁점

(1) 한국의 외교적 고립화

신 미·일방위협력지침에 대한 국내 논쟁 중 하나는 한국의 외교적 고립화 가능성입니다. 일부에서는 이 지침이 한국을 미국과 일본의 전략적 이익에 종속시키고, 중국과의 관계 악화를 초래하여 한국을 외교적으로 고립시킬 수 있다고 우려합니다.

이러한 우려는 특히 2015년 4월 22일 인도네시아 자카르타에서 열린 반둥회의를 계기로 더욱 강화되었습니다. 이 회의에서 중국과 일본은 정상회담을 개최하며 관계 개선의 모습을 보였습니다. 반면, 한국은 일본과의 갈등 관계를 유지하면서도 실질적인 성과를 거두지 못하고 있다는 평가를 받았습니다.

결과적으로, 신 미·일방위협력지침이 한국의 외교적 고립화를 초래할 수 있다는 우려는 다음과 같은 요인들에 근거합니다.

중·일 관계 개선: 2015년 반둥회의를皮切り로 중국과 일본은 관계 개선 움직임을 보이고 있습니다. 이는 한국이 미국과 일본 편에 서면서 중국과의 관계 악화를 초래할 수 있다는 우려를 불러일으켰습니다.
외교 전략 부재: 일부에서는 한국 정부가 미국, 일본, 중국과의 관계를 효과적으로 관리할 수 있는 명확한 외교 전략을 가지고 있지 못하다는 지적을 합니다. 이는 한국이 국익을 제대로 챙기지 못하고 외교적으로 고립될 수 있다는 우려로 이어집니다.
미·일과의 갈등 심화: 신 미·일방위협력지침이 실행될 경우, 한국은 미국과 일본과의 군사 협력을 강화하게 됩니다. 하지만 이는 중국과의 관계를 더욱 악화시키고, 한반도의 긴장을 고조시킬 수 있다는 우려가 있습니다.
따라서 신 미·일방위협력지침은 한국의 외교적 안보에 중요한 영향을 미칠 수 있는 정책으로서, 국내에서는 다양한 논의와 우려가 제기되고 있습니다.
"""

# Call the function to display the example texts
print_text_comparison(ground_truth_text, llama_text, gpt4_text, gpt35_text, clova_text, gemini_text)

Ground Truth:
반둥회의

Llama:
중일 정상회담이 성사된 것은 2015년 4월 22일 인도네시아 자카르타에서 개최된 반둥회의를 계기로 발생한 중일관계의 정상화 분위기 때문입니다.

GPT-4:
한국의 외교적 고립에 대한 염려를 더 크게 한 중일 정상회담은 2015년 4월 22일 인도네시아 자카르타에서 개최된 반둥회의를 계기로 성사되었습니다. 이 회의를 통해 중·일 간의 관계가 정상화되는 분위기가 형성되었고, 이는 한국이 일본과의 갈등을 유지하고 있음에도 불구하고 외교적으로 실질적인 성과를 보지 못하는 상황과 맞물려 한국의 외교적 고립화에 대한 우려를 증폭시켰습니다.

GPT-3.5:
중일 정상회담은 2015년 4월 22일에 개최된 반둥회의를 계기로 이루어졌어요. 이 회의를 통해 중일 관계가 일부 정상화되는 분위기가 형성되면서 정상회담이 이루어졌습니다.

Clova:
한국의 외교적 고립에 대한 염려를 더 크게 한 중일 정상회담은 2015년 4월 22일 인도네시아 자카르타에서 개최된 반둥회의를 계기로 성사되었습니다. 이 회담은 중·일관계가 정상화되는 듯한 분위기가 형성되면서 주목을 받았습니다.

Gemini:
4. 신 미·일방위협력지침의 주요 쟁점

(1) 한국의 외교적 고립화

신 미·일방위협력지침에 대한 국내 논쟁 중 하나는 한국의 외교적 고립화 가능성입니다. 일부에서는 이 지침이 한국을 미국과 일본의 전략적 이익에 종속시키고, 중국과의 관계 악화를 초래하여 한국을 외교적으로 고립시킬 수 있다고 우려합니다.

이러한 우려는 특히 2015년 4월 22일 인도네시아 자카르타에서 열린 반둥회의를 계기로 더욱 강화되었습니다. 이 회의에서 중국과 일본은 정상회담을 개최하며 관계 개선의 모습을 보였습니다. 반면, 한국은 일본과의 갈등 관계를 유지하면서도 실질적인 성과를 거두지 못하고 있다는 평가를 받았습니다.

결과적으로, 신 미·일방위협력지침이 한국의 외교적 고립화를 초래할 수 있다는 우려는 다음과 같은 요인들에 근거합니다.

중·일 관계 개선: 2015

In [ ]:
extracted_data[9]

{'context': '4. 신 미·일방위협력지침의 주요쟁점\n(1) 한국의 외교적 고립화\n신 미·일방위협력지침에 대해 국내 일각에서는 한국의 외교적 고립화를 주장하고 있다. 이러한 주장은 지난 2015년 4월 22일 인도네시아 자카르타에서 개최된 반둥회의를 계기로 중·일 정상회담이 개최되는 등 중·일관계도 정상화되는 듯한 분위기가 형성되면서 더욱 부각되고 있는데, 이는 한국이 일본과의 갈등관계를 유지하면서도 실제적인 성과가 없다는 판단에 따른 것이다. 즉 외교전략의 부재로 인해 대 미국과 대 중국 및 대 일본관계에서 한국의 국익을 제대로 챙기지 못하고 있다는 주장이다.\n',
 'question': '뭘 계기로 이루어진 중일 정상회담으로 대한민국은 외교적인 소외를 걱정하게 됐어',
 'answer': '반둥회의'}

In [ ]:
질문_prompt = format_context_question(extracted_data[9])
질문_prompt

'Context: 4. 신 미·일방위협력지침의 주요쟁점\n(1) 한국의 외교적 고립화\n신 미·일방위협력지침에 대해 국내 일각에서는 한국의 외교적 고립화를 주장하고 있다. 이러한 주장은 지난 2015년 4월 22일 인도네시아 자카르타에서 개최된 반둥회의를 계기로 중·일 정상회담이 개최되는 등 중·일관계도 정상화되는 듯한 분위기가 형성되면서 더욱 부각되고 있는데, 이는 한국이 일본과의 갈등관계를 유지하면서도 실제적인 성과가 없다는 판단에 따른 것이다. 즉 외교전략의 부재로 인해 대 미국과 대 중국 및 대 일본관계에서 한국의 국익을 제대로 챙기지 못하고 있다는 주장이다.\n\nQuestion: 뭘 계기로 이루어진 중일 정상회담으로 대한민국은 외교적인 소외를 걱정하게 됐어'

In [ ]:
llama3_inference_result = generate_response(system_message="너는 주어진 Context를 토대로 Question에 답변하는 챗봇이야. Question에 대한 답변을 한국어로 답해줘.",
                             user_message=질문_prompt)
print(llama3_inference_result)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


인도네시아 자카르타에서 개최된 반둥회의를 계기로 중·일 정상회담이 개최되는 듯한 분위기가 형성되었습니다.


In [ ]:
# 대표 LLM별 번역 추론 성능 비교
ground_truth_text = extracted_data[9]['answer']
llama_text = llama3_inference_result
gpt4_text = "중일 정상회담은 2015년 4월 22일 인도네시아 자카르타에서 개최된 반둥회의를 계기로 이루어졌습니다. 이로 인해 중·일관계가 정상화되는 분위기가 형성되면서 대한민국은 외교적인 소외를 걱정하게 되었습니다."
gpt35_text = "대한민국이 외교적인 소외를 걱정하게 된 계기는 중일 정상회담이 이루어진 후였어요. 이 회담은 2015년 4월 22일 인도네시아 자카르타에서 반둥회의를 계기로 진행되었고, 이때 중일 관계가 일부 정상화되는 분위기가 형성되었어요. 그런데 이런 과정에서 대한민국은 자신의 외교전략이 부족하다는 인식을 하게 되었고, 일본과의 갈등 관계 속에서도 실질적인 성과를 이루지 못한다는 판단이 나타났어요. 그 결과, 대한민국은 미국과 중국, 그리고 일본과의 관계에서 외교적으로 소외되는 상황에 대해 우려하게 되었죠."
clova_text = "중·일 정상회담은 지난 2015년 4월 22일 인도네시아 자카르타에서 개최된 반둥회의를 계기로 이루어졌습니다. 이 회담을 계기로 중·일관계가 정상화되는 듯한 분위기가 형성되면서 한국의 외교적 고립화에 대한 우려가 제기되었습니다. 이는 한국이 일본과의 갈등관계를 유지하면서도 실제적인 성과가 없다는 판단에 따른 것으로, 외교전략의 부재로 인해 대 미국과 대 중국 및 대 일본관계에서 한국의 국익을 제대로 챙기지 못하고 있다는 주장이 제기되었습니다."
gemini_text = """신 미·일 방위협력지침과 대한민국 외교적 고립화: 중일 정상회담이 촉발한 우려
1. 배경: 2015년 반둥회의와 중·일 관계 정상화 분위기

2015년 4월 22일 인도네시아 자카르타에서 개최된 반둥회의를 계기로 중·일 정상회담이 개최되었습니다. 이는 중·일 관계 정상화를 향한 움직임으로 해석되었으며, 이후 양국은 협력을 강화하는 모습을 보였습니다.

2. 한국의 우려: 외교적 고립화 가능성

이러한 상황 속에서 한국 일각에서는 신 미·일 방위협력지침으로 인해 한국이 외교적으로 고립될 가능성을 우려하기 시작했습니다. 특히, 다음과 같은 두 가지 요인이 주요 쟁점으로 떠올랐습니다.

한·미·일 삼각 동맹 구축 논의: 일부에서는 신 지침이 한·미·일 삼각 동맹 구축으로 이어져 한국이 미국과 일본의 전략에 종속될 수 있다고 우려했습니다. 이는 한국의 외교적 자율성을 제한하고, 중국과의 관계 악화로 이어질 수 있다는 주장입니다.
역사 문제: 과거 역사 문제로 인해 한국과 일본 간에는 갈등이 지속되고 있습니다. 신 지침이 일본의 군사적 역할 강화를 초래할 경우, 한반도의 긴장 고조와 역사 문제 해결의 어려움을 야기할 수 있다는 우려가 제기되었습니다.
3. 한국 정부의 입장: 균형 외교 노력 강조

이러한 우려에 대해 한국 정부는 균형 외교를 통해 미국, 일본, 중국과의 관계를 모두 유지하면서 국익을 추구할 것이라고 입장을 밝혔습니다. 또한, 신 지침은 한반도의 평화와 안정을 위한 방위 협력을 강화하는 데 기여한다고 강조했습니다.

4. 지속적인 논의와 평가 필요

신 미·일 방위협력지침이 한국의 외교적 입장에 미치는 영향은 아직 명확하게 드러나지 않았습니다. 향후 양국 관계와 한반도 상황 변화에 따라 지속적인 논의와 평가가 필요할 것으로 예상됩니다.
"""

# Call the function to display the example texts
print_text_comparison(ground_truth_text, llama_text, gpt4_text, gpt35_text, clova_text, gemini_text)

Ground Truth:
반둥회의

Llama:
인도네시아 자카르타에서 개최된 반둥회의를 계기로 중·일 정상회담이 개최되는 듯한 분위기가 형성되었습니다.

GPT-4:
중일 정상회담은 2015년 4월 22일 인도네시아 자카르타에서 개최된 반둥회의를 계기로 이루어졌습니다. 이로 인해 중·일관계가 정상화되는 분위기가 형성되면서 대한민국은 외교적인 소외를 걱정하게 되었습니다.

GPT-3.5:
대한민국이 외교적인 소외를 걱정하게 된 계기는 중일 정상회담이 이루어진 후였어요. 이 회담은 2015년 4월 22일 인도네시아 자카르타에서 반둥회의를 계기로 진행되었고, 이때 중일 관계가 일부 정상화되는 분위기가 형성되었어요. 그런데 이런 과정에서 대한민국은 자신의 외교전략이 부족하다는 인식을 하게 되었고, 일본과의 갈등 관계 속에서도 실질적인 성과를 이루지 못한다는 판단이 나타났어요. 그 결과, 대한민국은 미국과 중국, 그리고 일본과의 관계에서 외교적으로 소외되는 상황에 대해 우려하게 되었죠.

Clova:
중·일 정상회담은 지난 2015년 4월 22일 인도네시아 자카르타에서 개최된 반둥회의를 계기로 이루어졌습니다. 이 회담을 계기로 중·일관계가 정상화되는 듯한 분위기가 형성되면서 한국의 외교적 고립화에 대한 우려가 제기되었습니다. 이는 한국이 일본과의 갈등관계를 유지하면서도 실제적인 성과가 없다는 판단에 따른 것으로, 외교전략의 부재로 인해 대 미국과 대 중국 및 대 일본관계에서 한국의 국익을 제대로 챙기지 못하고 있다는 주장이 제기되었습니다.

Gemini:
신 미·일 방위협력지침과 대한민국 외교적 고립화: 중일 정상회담이 촉발한 우려
1. 배경: 2015년 반둥회의와 중·일 관계 정상화 분위기

2015년 4월 22일 인도네시아 자카르타에서 개최된 반둥회의를 계기로 중·일 정상회담이 개최되었습니다. 이는 중·일 관계 정상화를 향한 움직임으로 해석되었으며, 이후 양국은 협력을 강화하는 모습을 보였습니다.

2. 한국의 우려: 외교적 고립화 가능성

이러한 상황 속에서 한

# 추론 정확도에 대한 정량적 분석

In [ ]:
# LLama 3 : 100% (10/10)
# GPT-4 : 100% (10/10)
# GPT-3.5 : 90% (9/10)
# Clova : 100% (10/10)
# Gemini : 100% (10/10)